In [78]:
# 시스템 모듈
import os
from dotenv import load_dotenv

# 랭체인 모듈
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.vectorstores.faiss import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory 
from langchain_core.prompts import MessagesPlaceholder

# 환경변수 로드
load_dotenv()

# 벡터 DB 설정
DB_INDEX = "MANUAL_DB"
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.load_local(DB_INDEX, embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# LLM 모델 설정
llm = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0,
    streaming=True,
)

# 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages([
    ("system",
    """
    Answer the question using ONLY the following context.
    If you don't know the answer just say you don't know. DON'T make anything up.

    Context: {context}
    """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

# 검색 증강 문서 처리
def formatting_docs(docs):
    unique_sources = set()
    context = []
    for doc in docs:
        source = doc.metadata.get("source", "")
        if source not in unique_sources and os.path.exists(source):
            unique_sources.add(source)
            with open(source, 'r', encoding='utf-8') as file:
                content = file.read()
                context.append(content)
    
    return "\n\n".join(context)

# 대화 메모리 설정
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

# 체인 설정
chain = {
    "context": retriever | RunnableLambda(formatting_docs),
    "chat_history": RunnableLambda(lambda _: memory.load_memory_variables({}).get('chat_history', [])),
    "question": RunnablePassthrough()
} | prompt | llm 

# 대화 메모리를 사용한 입력 및 출력 관리
def run_chain_with_memory(query):
    # 체인을 사용하여 응답 생성
    response = chain.invoke(query)
    # 대화 메모리에 응답 추가
    memory.save_context(
        inputs={"human": query},
        outputs={"ai": response.content}
    )
    return response


In [79]:
query = "git이 무엇인가요?"
response = run_chain_with_memory(query)

print(response)

content='Git은 Linus Torvalds가 2005년에 개발한 분산 버전 관리 시스템(DVCS)입니다. 소프트웨어 개발 프로세스에서 소스 코드의 변경사항을 효과적으로 추적하고 관리하는 데 사용됩니다. Git의 주요 특징으로는 분산 버전 관리, 강력한 브랜칭 및 병합, 데이터 무결성, 스테이징 영역, 그리고 빠른 성능 등이 있습니다.' response_metadata={'finish_reason': 'stop'} id='run-7b7cf7ca-c42d-4866-8814-85a297251965-0'


In [80]:
query = "주요 특징이 무엇인가요"
response = run_chain_with_memory(query)

print(response)

content='Git의 주요 특징은 다음과 같습니다:\n\n1. **분산 버전 관리**: 각 개발자가 전체 저장소의 복사본을 로컬에 가지고 있어, 네트워크 연결 없이도 작업할 수 있습니다.\n\n2. **브랜치 및 병합**: Git의 강력한 브랜칭 모델을 통해 여러 개발 라인을 동시에 관리할 수 있습니다.\n\n3. **데이터 무결성**: Git은 SHA-1 해시를 사용하여 모든 커밋과 파일의 무결성을 보장합니다.\n\n4. **스테이징 영역**: 변경사항을 커밋하기 전에 검토하고 구성할 수 있는 중간 영역을 제공합니다.\n\n5. **빠른 성능**: Git은 대부분의 작업을 로컬에서 수행하므로 매우 빠릅니다.' response_metadata={'finish_reason': 'stop'} id='run-50f122d0-b983-413c-821f-862ca99b5f8d-0'


In [77]:
print(memory.load_memory_variables({}).get('chat_history'))



[HumanMessage(content='김장호가 누구인가요?'), AIMessage(content='죄송하지만, 제공된 정보에는 김장호에 대한 언급이 없습니다. 추가 정보가 필요하다면 다른 출처를 참고하시기 바랍니다.'), HumanMessage(content='git이 무엇인가요?'), AIMessage(content='Git은 Linus Torvalds가 2005년에 개발한 분산 버전 관리 시스템(DVCS)입니다. 소프트웨어 개발 프로세스에서 소스 코드의 변경사항을 효과적으로 추적하고 관리하는 데 사용됩니다. 주요 특징으로는 분산 버전 관리, 강력한 브랜칭 및 병합 기능, 데이터 무결성 보장, 스테이징 영역 제공, 그리고 빠른 성능 등이 있습니다. Git은 중앙 집중식 버전 관리 시스템과 달리 오프라인 작업이 가능하고, 더 빠른 브랜칭 및 병합을 지원하며, 전체 프로젝트 히스토리에 대한 로컬 액세스를 제공합니다.'), HumanMessage(content='내가 방금전에 어떤 내용을 물어보았나요?'), AIMessage(content='당신은 "김장호가 누구인가요?"라고 물어보셨습니다.')]
